In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, asc
from pyspark.sql.functions import when

spark = SparkSession.builder.appName("MallCustomers").getOrCreate()

customer = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("file:///SparkProject/Mall_Customers.csv")
    
print("Here is our inferred schema:")
customer.printSchema()


print("Group by gender and find average spending score")
customer.groupBy("Genre").avg("Score").show()

print("Filter out customers having spending score greater than 50")
customer.filter(customer['Score'] > 50).show(50)

print("Average spending score for  people having age less than 35")

Cubelow35 = customer.filter(customer['Age'] < 35)

AvgSpendingScore = Cubelow35.groupBy().avg("Score").show()


print("Average annual income for customers having spending score greater than 50")

CuScoreabove50 = customer.filter(customer['Score'] > 50)

AvgIncome = CuScoreabove50.groupBy().avg("Annual Income (k$)").show()

print("Top 10 customers with highest spending score")

cutop10 = customer.orderBy(customer.Score.desc()).show(10)

print("Top 10 customers with lowest spending score")

cubottom10 = customer.orderBy(customer.Score.asc()).show(10)

print("Group by age and find average spending score")
customer.groupBy("Age").avg("Score").show()



print("Group by age class and average spending score for each class.")

AgeGroups = customer.withColumn(
    "AgeGroup",
    when((customer["Age"] >= 18) & (customer["Age"] <= 28), '18-28')
    .when((customer["Age"] >= 29) & (customer["Age"] <= 38), '29-38')
    .when((customer["Age"] >= 39) & (customer["Age"] <= 48), '39-48')
    .when((customer["Age"] >= 49) & (customer["Age"] <= 58), '49-58')
    .when((customer["Age"] >= 59) & (customer["Age"] <= 71), '59-71')
    .otherwise('Unknown')
    )

GroupedandAvgSpend = AgeGroups.groupBy("AgeGroup").avg("Score")
GroupedandAvgSpendSorted = GroupedandAvgSpend.sort("avg(Score)")
Result = GroupedandAvgSpendSorted.show()



# Register the DataFrame as a SQL temporary view
customer.createOrReplaceTempView("customers")

print("Customer with maximum spending score")

maxSQL = spark.sql("SELECT * FROM customers WHERE Score = (SELECT max(Score)FROM customers)")
maxSQL.show()

print("Customer with minimum spending score")
minSQL = spark.sql("SELECT * FROM customers WHERE Score = (SELECT min(Score)FROM customers)")
minSQL.show()

spark.stop()

Here is our inferred schema:
root
 |-- CustomerID: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Income (k$): integer (nullable = true)
 |-- Score: integer (nullable = true)

Group by gender and find average spending score
+------+------------------+
| Genre|        avg(Score)|
+------+------------------+
|Female|51.526785714285715|
|  Male| 48.51136363636363|
+------+------------------+

Filter out customers having spending score greater than 50
+----------+------+---+------------------+-----+
|CustomerID| Genre|Age|Annual Income (k$)|Score|
+----------+------+---+------------------+-----+
|         2|  Male| 21|                15|   81|
|         4|Female| 23|                16|   77|
|         6|Female| 22|                17|   76|
|         8|Female| 23|                18|   94|
|        10|Female| 30|                19|   72|
|        12|Female| 35|                19|   99|
|        14|Female| 24|                20| 